# Curvature

Both convexity and the curvature distribution are computed from the same surface. For a molecular dynamics simulation, the creation of that surface is computationally expensive, relatively speaking. It is likely best to compute convexity and the curvature distribution together. Our experience was that creating the mesh surface accounted for roughly half of the computational time, with the calculation of the curvature distribution taking up most of the other half. 

In any case, we recommend running a test of just a few frames to get the timing information and determine if you need to run the calculation in parallel or not. 

Finally, this code relies heavily on the PyTim package. This is the package that can compute the Willard-Chandler surface and turn a collection of atoms into an analyzable mesh surface. However, the PyTim package only seems to work on Linux and Mac operating systems. We're looking into re-writing the PyTim package to interface with Windows systems but that is not implemented at this time. There is, of course, the possibility that there's a required dependency for Windows that I have not properly adjusted for, so you can try investigating that option as well. 

## File Size

The other issue to be considered with this analysis is the amount of data you generate. For our relatively small micelles, the typical mesh had 1-3,000 points and this code will compute curvature at every point. This means that for a decent sized trajectory, you are computing and writing to file several million to tens of millions of data points for txt files in the Gb file size ranges. There are a couple of options you may wish to explore to help reduce the size of these files. 

First, one of our biggest discoveries about the curvature distribution was that for the typical MD simulation where stoichiometry is set, the constant volume of your object keeps the curvature distribution relatively static. You can almost certainly get away with radically reduced sampling in time. Computing the curvature every 100 ps is probably a good high accuracy target, and every 500 ps-1 ns is probably a perfectly fine sampling rate. We have only tested these on reverse micelles, specifically a system with only one, pre-built reverse micelle, so we cannot guarantee this works for every system. However, it should work for most systems, especially if you meet the constant volume condition.

Second, a major issue with the curvature distributions is the creation of outliers. We do not have a real, curved surface and instead have small sections of straight lines approximating a curved surface. The curvature will only ever be estimated and even the best algorithm will occassionally produce erroneously large values. Removing these outliers will at least slightly reduce the amount of data involved. We are working on producing a function to do this, but in the mean time, that work falls on you.

Third, the most likely thing you'll do with this data is take a histogram of the values at each time point. If that is what you intend to do with it, then you can take a histogram before writing to file and write the histogram to file instead of the raw values. For even a large number of bins, you should be reducing the number of data entries by at least an order of magnitude this way.


In [ ]:
# Prereq Packages
import numpy as np
import MDAnalysis as mda
from scipy import spatial
import pytim
import pyvista as pv
from pytim.datafiles import *

## The Willard-Chandler Surface

The interface needed to create a Willard-Chandler surface of the micelle is relatively simple. We are demonstrating it's use with a trajectory loaded via MDAnalysis. Check out PyTim's GitHub page for example usage in other interfaces using the link below.

https://github.com/Marcello-Sega/pytim

We specifically modelled using the GROMACS package and those are the file types we're showing here. See MDAnalysis documentation for other acceptable file types.

https://userguide.mdanalysis.org/stable/examples/quickstart.html


In [ ]:
# Importing your trajectory as a universe object

# I prefer to set file names here for easy changing and reading
Path = 'Path/To/Files/'
Top = 'Your_Topology.gro'     #"lists atoms, residues, and their connectivity"
Traj = 'Your_Trajectory.xtc'  #'contains a list of coordinates in the order defined in the topology'

# creating a universe object
# equivalently, loading the trajectory into MDAnalysis
u = mda.Universe(Path+Top,Path+Traj)

# now we need to select the atoms that will define our surface
# MDAnalysis has a rich atom selection language
# Here is the selection used for our micelles
# Selection includes the water interior as well as the hydrophilic head group
# of AOT up to the ester oxygens
Core_Selection = '(resname AOT and (type O or type S or name C1 or name H1 or name C2 or name H2 or name H3 or name C3 or name C12)) or resname SOL'

# Now to actually create a selection
Core = u.select_atoms(Core_Selection)


## Radii Dictionary

The Willard-Chandler surface defines a continuous surface "as the isodensity surface of a Gaussian kernel density estimate". As such, it needs to know what Gaussian estimate of electron density to use for each type of atom. PyTim has default values set up for the CHARMM27, Amber03, and G43a1, but you will need to build your own dictionary for other force fields.

The main purpose is just to define where the atoms your interested in are, so small changes or errors in the Gaussian electron density should not impact your shape significantly. A simple copy of one of the pre-built dictionaries with a similar atom type mapping should be sufficient. PyTim may have a function to create this type of map for an arbitrary force field and we will look into adapting that into a useable function in the future. 

Reference:

Sega, M.; Hantal, G.; Fabian, B.; Jedlovszky, P.; Pytim: A Python Package for the Interfacial Analysis of Molecular Simulations. *J. Comp. Chem.*,**2018**, *39*. 2118-2125.

Willard Chandler Documentation:

https://marcello-sega.github.io/pytim/WillardChandler.html

In [ ]:
'''
Creating the Willard-Chandler surface

The first argument is your universe object

"group" is the selection of atoms you're interested in

Alpha will control the precision that the Gaussian density functions are
computed out to. 3 should be more than adequate.

The mesh value will control the grid spacing of points the volume is sampled
on. A smaller number produces a smoother surface with more points on it at the
cost of dramatically increased computational time (~O(N^2))
1.1 seems perfectly adequate for shape analyses. We used 0.5 to produce cleaner
surfaces on the example micelle images in Figure 4 of our paper, but it took long
enough that we do not recommend this for analyzing a full trajectory

"fast" gives the option to use a faster version with truncated Gaussians. We did
not experiment much with this option and opted not to use it. Feel free to set
it to your preference. 
'''
WC = pytim.WillardChandler(u,group=Core,alpha=3.0,
                           mesh=1.1,fast=False,
                           radii_dict=pytim_data.vdwradii(CHARMM27_TOP))


## Mesh I/O Format

The actual creation of the Willard-Chandler surface is very easy and convenient with PyTim. The only catch is the output. We are using PyVista as a powerful and convenient package to handle manipulating, analyzing and visualizing the mesh. However, PyVista requires that the output be in a specific format. 

A typical mesh output usually contains 2 arrays. The first is just a list of the x, y, and z coordinates of every point. The second is a list of how those points connect to create the surface. Usually, that second list is of a form like:

>[1,4,3]\
[1,6,9,2]

etc. Where each number is the index of the point from the list of xyz coordinates. PyTim creates a Delauney triangulated mesh where every face on the surface is made up of only 3 points. However, in general a mesh can have faces of any number of vertices. So PyTim spits out a nice, concise array with only 3 columns, but PyVista requires a format that is more general to accomodate any number of vertices in a face. They do this by creating a flattened (1D) array where the first number denotes the number of vertices for that face, then the index number of each vertex in that face, then the number of vertices for the next face, and so on. 

This is all a long way to say that this is why the code looks a bit more complicated than it needs to be. We're just adjusting formats from PyTim to PyVista styles.

In [ ]:
# adjusting formats

# getting vertex xyz coord array and face connectivity array
verts = WC.triangulated_surface[0]
faces = WC.triangulated_surface[1]

# creating blank list of 3s to match PyVista formatting
threes = 3*np.ones((faces.shape[0],1),dtype=int)

# converting
faces = np.concatenate((threes,faces),axis=1)

# creating a PyVista PolyData object to handle the mesh
Poly = pv.PolyData(verts,faces)


In [ ]:
'''
Computing Curvature

PyVista have a function to compute the curvature values by default
This will compute the curvature at every vertex in your mesh,
which can be an immense amount of data. While simple, this will be
one of the most computationally expensive portions of the code.

The data I write to file in the python files includes all points in case you
want to do a more nuanced method of aggregating the data. However, you could
dramatically reduce the file size by taking the histogram of the curvature
values and only writing those results to file (for micelles, we had 1-3,000
points while a good histogram only had 100 bins for 200 items written to file)
'''

mean_curv = Poly.curvature(curv_type='mean')
G_curv = Poly.curvature(curv_type='Gaussian')
